# Package Installation and Data Reading

In [6]:
import os
import numpy as np
import pandas as pd

In [21]:
def read_csv_files_to_dataframes(relative_path: str) -> dict:
    """
    Read CSV files from a specified relative path and store them in separate DataFrames.

    Args:
        relative_path (str): The relative path of the directory containing the CSV files.

    Returns:
        dict: A dictionary where keys are the file names (without extension) and values are the corresponding DataFrames.

    Raises:
        FileNotFoundError: If the specified relative path does not exist.

    Example:
        dataframes = read_csv_files_to_dataframes('data_folder')
        df1 = dataframes['file1']  # Access the DataFrame for file1.csv
        df2 = dataframes['file2']  # Access the DataFrame for file2.csv
    """
    data_folder = os.path.join(os.getcwd(), relative_path)

    if not os.path.exists(data_folder):
        raise FileNotFoundError(f"The specified relative path '{relative_path}' does not exist.")

    csv_files = [file for file in os.listdir(data_folder) if file.endswith('.csv')]
    dataframes = {}

    for file in csv_files:
        file_path = os.path.join(data_folder, file)
        df_name = os.path.splitext(file)[0]
        dataframes[df_name] = pd.read_csv(file_path)

    return dataframes


In [22]:
dataframes = read_csv_files_to_dataframes("/Users/alexholzer/Desktop/retail_price_prediction/data")

In [25]:
for file_name in dataframes:
    print(file_name)    


file_out2
file_out


In [20]:
dataframes["file_out2"]

,Unnamed: 0,InvoiceID,Date,ProductID,TotalSales,Discount,CustomerID,Quantity
0,0,328,2019-12-27,1684,796.610169,143.389831,185,4
1,1,329,2019-12-27,524,355.932203,64.067797,185,2
2,2,330,2019-12-27,192,901.694915,162.305085,230,4
3,3,330,2019-12-27,218,182.754237,32.895763,230,1
4,4,330,2019-12-27,247,780.101695,140.418305,230,4
...,...,...,...,...,...,...,...,...
29098,29098,11092,2023-01-13,1644,6573.000000,1183.140000,269,1
29099,29099,11093,2023-01-13,352,5179.728814,932.351186,250,4
29100,29100,11094,2023-01-13,683,7741.423729,1393.456271,415,4
29101,29101,11095,2023-01-14,1830,3644.067797,655.932203,59,4
